# Structural measurements on kinases

1. dataclass for loading and working with data
2. functions for systematically generating data for multiple targets
3. application and results

## Code

### Imports

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import re
import os
from sympy import Plane, Point3D
import copy

import matplotlib as mpl
mpl.style.use('ggplot')
import matplotlib.pyplot as plt



/Users/matthias/Documents/hibit.in/development/dev_env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Dataclass

In [8]:

class molData:
    def __init__(self,path,data_section = 0):
        my_data = pd.read_table(path,skiprows=8,header=None,delim_whitespace=True,index_col=0)
        atoms = my_data.loc[:"@<TRIPOS>BOND"].iloc[:-1]
        atoms.columns = ["atomID","x","y","z","compound2",
                         "residueNR","residue","charge","mainChain"]
        resPattern = r"[A-Z]{3}[0-9]*"
        hohPattern = r"HOH[0-9]*"
        residueMask = atoms.loc[:,"residue"].str.match(resPattern,as_indexer=True).values
        hohMask = atoms.loc[:,"residue"].str.match(hohPattern,as_indexer=True).values
        self.other = atoms.loc[np.logical_not(residueMask)]
        self.water = atoms.loc[hohMask]
        self.chain = atoms.loc[np.logical_and(residueMask,np.logical_not(hohMask))]
        resDigest = self.chain.loc[:,"residue"].str.extract('(?P<residueType>[A-Z]{3})(?P<residueID>[0-9]*)')
        self.chain = self.chain.join(resDigest)
        self.chainIDGroups = self.chain.groupby(["residueID","atomID"])
        self.chainNRGroups = self.chain.groupby(["residueNR","atomID"])
        self.bonds = my_data.loc["@<TRIPOS>BOND":"@<TRIPOS>SUBSTRUCTURE"].iloc[1:-1]
        self.subst = my_data.loc["@<TRIPOS>SUBSTRUCTURE":].iloc[1:-2]

    def anchordistance(self,res1,res2): # fix
        coord1 = self.old_get_atom_coords(res1)
        coord2 = self.old_get_atom_coords(res2)
        return self.distance(coord1,coord2)
        
    def distance(self,coord1,coord2):
        return np.linalg.norm(coord1 - coord2)
    
    def old_get_atom_coords(self, residueID, anchor_atom=2):
        """
            extracts coordinates for target atom according to residue type 
        """
        residue = self.chainIDGroups.get_group((residueID))
        anchor = residue.iloc[anchor_atom]
        coords = anchor.loc["x":"z"]
        return np.array(coords).astype(np.float32)
    
    def get_atom_coords1(self, residueID, anchor_atom="CA"): # combine with 2 later
        anchor = self.chainIDGroups.get_group((residueID,anchor_atom))
        coords = anchor.loc[:,"x":"z"]
        return np.array(coords).astype(np.float32)

    def get_atom_coords2(self, residueNR, anchor_atom="CA"):
        try:
            anchor = self.chainNRGroups.get_group((residueNR,anchor_atom))
        except KeyError as e:
            e.args += (residueNR,anchor_atom,self.chainNRGroups.groups, )
            raise
        coords = anchor.loc[:,"x":"z"]
        result = np.array(coords.values).astype(np.float32)[0]
        return result
    
    def get_residue_info(self, residueNR):
        anchor = self.chainNRGroups.get_group((residueNR,"CA"))
        residueType = anchor.loc[:,"residueType"].values
        residueID = anchor.loc[:,"residueID"].values
        return (residueID, residueType)
        
    def chi_angle(self,coords):
        vector1 = np.subtract(coords[1],coords[0])
        vector2 = np.subtract(coords[2],coords[1])
        vector3 = np.subtract(coords[3],coords[2])
        normal1 = np.cross(vector1,vector2)
        normal2 = np.cross(vector2,vector3)
        normNormal1 = np.divide(normal1,np.linalg.norm(normal1))
        normNormal2 = np.divide(normal2,np.linalg.norm(normal2))
        dot1 = np.dot(normNormal1,normNormal2)
        cross1 = np.cross(normNormal1,normNormal2)
        # calculation of sign by ((v1xv2)xv2)*(v2xv3)
        sign = np.sign(np.dot(np.cross(vector2,normal1),normal2))
        if sign == 0: sign = 1
        angle1 = sign*np.arccos(dot1)
        return angle1
        
    def calculate_chi1(self,residue):
        Amcrd = self.get_atom_coords2(residue, "N")
        CAcrd = self.get_atom_coords2(residue, "CA")
        CBcrd = self.get_atom_coords2(residue, "CB")
        CGcrd = self.get_atom_coords2(residue, "CG")
        coords = (Amcrd, CAcrd, CBcrd, CGcrd)
        return self.chi_angle(coords)
    
    def calculate_chi2(self,residue):
        CAcrd = self.get_atom_coords2(residue, "CA")
        CBcrd = self.get_atom_coords2(residue, "CB")
        CGcrd = self.get_atom_coords2(residue, "CG")
        SDcrd = self.get_atom_coords2(residue, "SD")
        coords = (CAcrd, CBcrd, CGcrd, SDcrd)
        return self.chi_angle(coords)
    
    def chi_angles(self,residue):
        chi1 = self.calculate_chi1(str(residue))
        chi2 = self.calculate_chi2(str(residue))
        return [chi1, chi2]

    

### Mass calculation method

In [33]:

"""
bad implementation of key mapping
"""

DATASECTIONS = {
    "complex": "/complex.mol2",
    "pocket": "/pocket.mol2"
}

MEASUREMENTS = {
    "distance": "distance",
    "angles": "chi_angles",
    "gatekeepers": "met_gk_id"
}

MEASID = {
    "distance": 0,
    "chi_angles": 1,
    "met_gk_id": 2
}

"""
function: loop through different kinase alternatives and perform measurements
"""
def calculate_kinase_data(structure_path, data_section, measurement, res_ids=None):
    structures = pd.Series([name for name in os.listdir(structure_path) if os.path.isdir(structure_path + name)])
    structure_labels = structures.str.replace("chain","").str.lower().str.replace("_alt[ab]","")
    results = pd.DataFrame(index = structure_labels)
    if measurement == 0:
        results["measured"] = pd.Series()
    elif measurement == 1:
        results["chi1"] = pd.Series()
        results["chi2"] = pd.Series()
    elif measurement == 2:
        results["resID"] = pd.Series()
    fails = []

    for structure, structure_label in zip(structures,structure_labels):
        myurl = structure_path + structure + DATASECTIONS[data_section]
        mydata = molData(myurl,data_section)

# DIFFERENT MEASUREMENTS
# 1 distances
        if measurement == 0:
            residueID1 = res_ids[0]
            residueID2 = res_ids[1]
            results.loc[structure_label,"measured"] = mydata.anchordistance(residueID1,residueID2)

# 2 chi angles
        elif measurement == 1:
            if res_ids is None:
                res_ids = 45
            residueID = res_ids
            try:
                angles = mydata.chi_angles(residueID)
                results.loc[structure_label,"chi1"] = angles[0]
                results.loc[structure_label,"chi2"] = angles[1]
            except KeyError as e:
                fails.append(structure_label)

# 3 gatekeeper
        elif measurement == 2:
            try:
                resID, resType = mydata.get_residue_info("45")
            except KeyError as e:
                fails.append(structure_label)
            if resType == "MET":
                results.loc[structure_label,"resID"] = resID
    return (results, fails)

"""
function: loop through different kinases and perform kinase data calculation function on each
"""
def calculate_organism_data(organism_path,data_section,measurement,res_ids=None):
    kinases = pd.Series([name for name in os.listdir(organism_path) if os.path.isdir(organism_path + name)])
    measurement = MEASUREMENTS[measurement]
    measurement = MEASID[measurement]
    if measurement == 0:
        results = pd.DataFrame(columns = ["measured"])
    elif measurement == 1:
        results = pd.DataFrame(columns = ["chi1","chi2"])
    elif measurement == 2:
        results = pd.DataFrame(columns = ["resID"])
    for kinase in kinases:
#        print "starting on " + kinase
        kinase_path = organism_path + kinase + "/"
        kinase_data = calculate_kinase_data(kinase_path, data_section, measurement, res_ids)
        results = results.append(kinase_data[0])
    return results


## Calculate methionine gatekeeper chi angles

In [4]:
myurl1 = "/Users/matthias/Documents/hibit.in/KLIFS_LigandBound2/HUMAN/"
myurl2 = "/Users/matthias/Documents/hibit.in/KLIFS_LigandUnBound2/HUMAN/"
mydata = "pocket"
mymeasurement = "angles"




In [49]:
dataBound = calculate_organism_data(myurl1, mydata, mymeasurement)


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [13]:
dataUnbound = calculate_organism_data(myurl2, mydata, mymeasurement)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [45]:
dataBound = pd.read_csv("/Users/matthias/Documents/hibit.in/development/KinStruct/dataBound.csv",
                        index_col = 0)

In [ ]:

dataUnbound = pd.read_csv("/Users/matthias/Documents/hibit.in/development/KinStruct/dataUnbound.csv",
                          index_col = 0)

In [50]:

# radians to degrees

dataBound.chi1 = dataBound.chi1.mul(180/np.pi)
dataBound.chi2 = dataBound.chi2.mul(180/np.pi)


In [ ]:


dataUnbound.chi1 = dataUnbound.chi1.mul(180/np.pi)
dataUnbound.chi2 = dataUnbound.chi2.mul(180/np.pi)

In [15]:
dataBound = dataBound.astype(np.float32).drop_duplicates()
dataUnbound = dataUnbound.astype(np.float32).drop_duplicates()

In [71]:
dataBound.to_csv("/Users/matthias/Documents/hibit.in/development/KinStruct/dataBound.csv")

In [72]:
dataUnbound.to_csv("/Users/matthias/Documents/hibit.in/development/KinStruct/dataUnbound.csv")

In [36]:
dataUnbound.dropna(axis=0).chi2.values
pass


In [ ]:
dataBound

In [66]:

angleChi1 = -180
angleChi2 = 0

dataBoundShift = dataBound.copy()
dataBoundShift.chi1[dataBound.chi1.lt(angleChi1)] = dataBound[dataBound.chi1.lt(angleChi1)].chi1.add(360)
dataBoundShift.chi2[dataBound.chi2.lt(angleChi2)] = dataBound[dataBound.chi2.lt(angleChi2)].chi2.add(360)

dataUnboundShift = dataUnbound.copy()
dataUnboundShift.chi1[dataUnbound.chi1.lt(angleChi1)] = dataUnbound[dataUnbound.chi1.lt(angleChi1)].chi1.add(360)
dataUnboundShift.chi2[dataUnbound.chi2.lt(angleChi2)] = dataUnbound[dataUnbound.chi2.lt(angleChi2)].chi2.add(360)


## Plot results

there is a weird swapping of axis happening, need to figure out this first

In [28]:

nonaBound = dataBound.dropna(axis=0)
nonaBoundchi1 = nonaBound.chi1
nonaBoundchi2 = nonaBound.chi2
nonaUnbound = dataUnbound.dropna(axis=0)
nonaUnboundchi1 = nonaUnbound.chi1
nonaUnboundchi2 = nonaUnbound.chi2
binsX = np.linspace(-180,180,50)
binsY = np.linspace(-180,180,50)
HBound,xedges,yedges = np.histogram2d(nonaBoundchi1.values,nonaBoundchi2.values,bins=(binsX,binsY))
HUnbound,xedges,yedges = np.histogram2d(nonaUnboundchi1.values,nonaUnboundchi2.values,bins=(binsX,binsY))
HBound = np.transpose(HBound)
HUnbound = np.transpose(HUnbound)

"""
fig = plt.figure()
ax = fig.add_subplot(121)
nonaBoundchi1.hist()
plt.title("chi1")
ax = fig.add_subplot(122)
nonaBoundchi2.hist()
plt.title("chi2")

fig = plt.figure()
ax = fig.add_subplot(121)
nonaUnboundchi1.hist()
plt.title("chi1")
ax = fig.add_subplot(122)
nonaUnboundchi2.hist()
plt.title("chi2")
"""

X,Y = np.meshgrid(xedges,yedges)


In [ ]:


ticks = np.linspace(-180,180,5)
tickLocator = mpl.ticker.FixedLocator(ticks)

fig1 = plt.figure(figsize = (7,7),dpi = 300)

ax1 = fig1.add_subplot(111)
ax1.pcolormesh(X,Y,HBound,cmap='magma_r')
ax1.xaxis.set_major_locator(copy.copy(tickLocator))
ax1.yaxis.set_major_locator(copy.copy(tickLocator))
ax1.xaxis.set_label_text(u"Chi 1 [°]")
ax1.yaxis.set_label_text(u"Chi 2 [°]")
plt.grid(b=True, which='major', color='0.65', linestyle='-')



fig2 = plt.figure(figsize = (7,7),dpi = 300)
ax2 = fig2.add_subplot(111)
ax2.pcolormesh(X,Y,HUnbound,cmap='magma_r')
ax2.xaxis.set_major_locator(copy.copy(tickLocator))
ax2.yaxis.set_major_locator(copy.copy(tickLocator))
ax2.xaxis.set_label_text(u"Chi 1 [°]")
ax2.yaxis.set_label_text(u"Chi 2 [°]")
plt.grid(b=True, which='major', color='0.65', linestyle='-')


fig1.savefig("/Users/matthias/Documents/hibit.in/development/KinStruct/hist2dBound")
fig2.savefig("/Users/matthias/Documents/hibit.in/development/KinStruct/hist2dUnbound")

plt.show()

In [ ]:

fig = plt.figure(figsize = (8,8),dpi = 300)
ax = fig.add_subplot(111)
ax = dataBound.plot.scatter("chi1","chi2",color="DarkBlue",label="Ligand Bound", ax=ax)
ax = dataUnbound.plot.scatter("chi1","chi2",color="DarkGreen",label="No Ligand", ax=ax)
ax.set_xlim(-180,180)
ax.set_ylim(-180,180)
ax.xaxis.set_label_text(u"Chi1 [˚]")
ax.yaxis.set_label_text(u"Chi2 [˚]")
fig.savefig("/Users/matthias/Documents/hibit.in/development/KinStruct/chi_angles1.png")

fig = plt.figure(figsize = (8,8), dpi = 300)
ax = fig.add_subplot(111)
ax = dataUnbound.plot.scatter("chi1","chi2",color="DarkGreen",label="No Ligand", ax=ax)
ax = dataBound.plot.scatter("chi1","chi2",color="DarkBlue",label="Ligand Bound", ax=ax)
ax.set_xlim(-180,180)
ax.set_ylim(-180,180)
fig.savefig("/Users/matthias/Documents/hibit.in/development/KinStruct/chi_angles2.png")

fig = plt.figure(figsize = (8,8),dpi = 300)
ax = fig.add_subplot(111)
ax = dataBoundShift.plot.scatter("chi1","chi2",color="DarkBlue",label="Ligand Bound", ax=ax)
ax = dataUnboundShift.plot.scatter("chi1","chi2",color="DarkGreen",label="No Ligand", ax=ax)
ax.set_xlim(-180,180)
ax.set_ylim(0,360)
ax.xaxis.set_label_text(u"Chi1 [˚]")
ax.yaxis.set_label_text(u"Chi2 [˚]")
fig.savefig("/Users/matthias/Documents/hibit.in/development/KinStruct/chi_angles_shift1.png")

fig = plt.figure(figsize = (8,8), dpi = 300)
ax = fig.add_subplot(111)
ax = dataUnboundShift.plot.scatter("chi1","chi2",color="DarkGreen",label="No Ligand", ax=ax)
ax = dataBoundShift.plot.scatter("chi1","chi2",color="DarkBlue",label="Ligand Bound", ax=ax)
ax.set_xlim(-180,180)
ax.set_ylim(0,360)
fig.savefig("/Users/matthias/Documents/hibit.in/development/KinStruct/chi_angles_shift2.png")

plt.show()

In [ ]:

xedges = np.linspace(0,np.pi,100)
yedges = np.linspace(0,np.pi,100)
X, Y = np.meshgrid(xedges, yedges)
cmap1 = plt.get_cmap('Blues')
cmap2 = plt.get_cmap('Blues')
ticks = np.linspace(0,np.pi,5)
tickLocator = mpl.ticker.FixedLocator(ticks)
tickFormatter = mpl.ticker.FormatStrFormatter('%.2f')

H1, xedges1, yedges1 = np.histogram2d(dataBound["chi1"].values,dataBound["chi2"].values,
                                      bins=(xedges,yedges))
H2, xedges2, yedges2 = np.histogram2d(dataUnbound["chi1"].values,dataUnbound["chi2"].values,
                                      bins=(xedges,yedges))

fig = plt.figure(figsize = (15,8),dpi = 600)
ax1 = fig.add_subplot(121)
ax1.set_title("Bound data")
ax1.pcolormesh(X,Y,H1,cmap=cmap1)
ax1.set_aspect('equal')
ax1.xaxis.set_major_locator(copy.copy(tickLocator))
ax1.yaxis.set_major_locator(copy.copy(tickLocator))
ax1.xaxis.set_major_formatter(tickFormatter)
ax1.yaxis.set_major_formatter(tickFormatter)
ax1.xaxis.set_label_text("Chi 1")
ax1.yaxis.set_label_text("Chi 2")
plt.autoscale(True)

ax2 = fig.add_subplot(122)
ax2.set_title("Unbound data")
ax2.pcolormesh(X,Y,H2,cmap=cmap2)
ax2.set_aspect('equal')
ax2.xaxis.set_major_locator(copy.copy(tickLocator))
ax2.yaxis.set_major_locator(copy.copy(tickLocator))
ax2.xaxis.set_major_formatter(tickFormatter)
ax2.yaxis.set_major_formatter(tickFormatter)
ax2.xaxis.set_label_text("Chi 1")
ax2.yaxis.set_label_text("Chi 2")
plt.autoscale(True)

fig.savefig("/Users/matthias/Documents/hibit.in/development/KinStruct/fig2.png")


In [ ]:

xedges = np.linspace(0,np.pi,100)
yedges = np.linspace(0,np.pi,100)
X, Y = np.meshgrid(xedges, yedges)
cmap1 = plt.get_cmap('Blues')
cmap2 = plt.get_cmap('YlOrBr')
ticks = np.linspace(0,np.pi,5)
tickLocator = mpl.ticker.FixedLocator(ticks)
tickFormatter = mpl.ticker.FormatStrFormatter('%.2f')

H1, xedges1, yedges1 = np.histogram2d(dataBound["chi1"].values,dataBound["chi2"].values,
                                      bins=(xedges,yedges))
H2, xedges2, yedges2 = np.histogram2d(dataUnbound["chi1"].values,dataUnbound["chi2"].values,
                                      bins=(xedges,yedges))

fig = plt.figure(figsize = (8,8),dpi = 600)
ax1 = fig.add_subplot(111)
ax1.set_title("overlay data")
ax1.pcolormesh(X,Y,H1,cmap=cmap1,alpha = 1)
ax1.pcolormesh(X,Y,H2,cmap=cmap2,alpha = 0.3)
ax1.set_aspect('equal')
ax1.xaxis.set_major_locator(copy.copy(tickLocator))
ax1.yaxis.set_major_locator(copy.copy(tickLocator))
ax1.xaxis.set_major_formatter(tickFormatter)
ax1.yaxis.set_major_formatter(tickFormatter)
ax1.xaxis.set_label_text("Chi 1")
ax1.yaxis.set_label_text("Chi 2")
plt.autoscale(True)

fig.savefig("/Users/matthias/Documents/hibit.in/development/KinStruct/overlay.png")


## gatekeeper residue ids

In [91]:
myurl1 = "/Users/matthias/Documents/hibit.in/KLIFS_LigandBound2/HUMAN/"
myurl2 = "/Users/matthias/Documents/hibit.in/KLIFS_LigandUnBound2/HUMAN/"
mydata = "pocket"
mymeasurement = "gatekeepers"


gatekeepersBound = calculate_organism_data(myurl1, mydata, mymeasurement)
gatekeepersUnbound = calculate_organism_data(myurl2, mydata, mymeasurement)


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [79]:
gatekeepersBound = pd.read_csv("/Users/matthias/Documents/hibit.in/development/KinStruct/gatekeepersBound.csv",index_col = 0,dtype='str')
gatekeepersUnbound = pd.read_csv("/Users/matthias/Documents/hibit.in/development/KinStruct/gatekeepersUnbound.csv", index_col = 0,dtype='str')

In [92]:
gatekeepersBound.loc[:,"alternatives"] = gatekeepersBound.index
gatekeepersUnbound.loc[:,"alternatives"] = gatekeepersUnbound.index

In [97]:
gatekeepersBound = gatekeepersBound.drop_duplicates()
gatekeepersUnbound = gatekeepersUnbound.drop_duplicates()


In [100]:
gatekeepersBound = gatekeepersBound.drop("alternatives",axis=1)
gatekeepersUnbound = gatekeepersUnbound.drop("alternatives",axis=1)

In [ ]:
gatekeepersBound

In [95]:
gatekeepersUnbound.resID = gatekeepersUnbound.resID.str.strip('[]\'').astype(np.float32)
gatekeepersBound.resID = gatekeepersBound.resID.str.strip('[]\'').astype(np.float32)
pass

In [102]:
gatekeepersBound.to_csv("/Users/matthias/Documents/hibit.in/development/KinStruct/gatekeepersBound.csv")
gatekeepersUnbound.to_csv("/Users/matthias/Documents/hibit.in/development/KinStruct/gatekeepersUnbound.csv")

# ## Old stuff

### Load excel file

In [8]:

#separate because takes time

reference = pd.read_excel("/Users/matthias/Documents/hibit.in/KinaseDomainMeasures-Novartis.xlsx",
                          header=0,
                          sheetname=0,
                          parse_cols="B,C,AJ")



### Compare calculated vs reference

(old stuff, currently doesn't work because of changes in dataclass atom referencing, ergo no calculation)

In [ ]:

ref_mod = reference.set_index("pdbchain",drop=False).loc[results.index]
ref_mod.loc[:,"pdbchain_unique"] = ref_mod.loc[:,"pdbchain_unique"].str.replace(r'[\S]{4}_[abe]_','')
ref_mod = ref_mod.drop_duplicates()
ref_mod = ref_mod.pivot(index="pdbchain",columns="pdbchain_unique",values="Ploop_bend6").iloc[1:,1:]
end_results = pd.concat([results,ref_mod],axis=1)

end_results["diff1"] = end_results["measured"].sub(end_results["pdb"],axis=0)
end_results["diff2"] = end_results["measured"].sub(end_results["pdbredo"],axis=0)
end_results